# [PyBroMo](http://tritemio.github.io/PyBroMo/) - 2. Generate smFRET data, including mixtures

<small><i>
This notebook is part of <a href="http://tritemio.github.io/PyBroMo" target="_blank">PyBroMo</a> a 
python-based single-molecule Brownian motion diffusion simulator 
that simulates confocal smFRET
experiments.
</i></small>

## *Overview*

*In this notebook we show how to generated smFRET data files from the diffusion trajectories*.

## Loading the software

Import all the relevant libraries:

In [ ]:
%matplotlib inline
from pathlib import Path
import numpy as np
import tables
import matplotlib.pyplot as plt
import seaborn as sns
import pybromo as pbm
print('Numpy version:', np.__version__)
print('PyTables version:', tables.__version__)
print('PyBroMo version:', pbm.__version__)

# Create smFRET data-files

## Create a file for a single FRET efficiency

In this section we show how to save a single smFRET data file. In the next section we will perform the same steps in a loop to generate a sequence of smFRET data files.

Here we load a diffusion simulation opening a file to save
timstamps in *write* mode. Use `'a'` (i.e. *append*) to keep 
previously simulated timestamps for the given diffusion.

In [ ]:
S = pbm.ParticlesSimulation.from_datafile('0168', mode='w')

In [ ]:
S.particles.diffusion_coeff_counts

In [ ]:
#S = pbm.ParticlesSimulation.from_datafile('0168')

## Simulate timestamps of smFRET

### Example1: single FRET population

Define the simulation parameters with the following syntax:

In [ ]:
params = dict(
    em_rates = (200e3,),    # Peak emission rates (cps) for each population (D+A)
    E_values = (0.75,),     # FRET efficiency for each population
    num_particles = (20,),  # Number of particles in each population
    bg_rate_d = 1500,       # Poisson background rate (cps) Donor channel
    bg_rate_a = 800,        # Poisson background rate (cps) Acceptor channel
    )

Create the object that will run the simulation and print a summary:

In [ ]:
mix_sim = pbm.TimestapSimulation(S, **params)
mix_sim.summarize()

Run the simualtion:

In [ ]:
rs = np.random.RandomState(1234)
mix_sim.run(rs=rs, overwrite=False, skip_existing=True)

Save simulation to a smFRET [Photon-HDF5](http://photon-hdf5.org) file:

In [ ]:
mix_sim.save_photon_hdf5(identity=dict(author='John Doe', 
                                       author_affiliation='Planet Mars'))

### Example 2: 2 FRET populations

To simulate 2 population we just define the parameters with 
one value per population, except for the Poisson background 
rate that is a single value for each channel.

In [ ]:
params = dict(
    em_rates = (200e3, 180e3),   # Peak emission rates (cps) for each population (D+A)
    E_values = (0.75, 0.35),     # FRET efficiency for each population
    num_particles = (20, 15),    # Number of particles in each population
    bg_rate_d = 1500,       # Poisson background rate (cps) Donor channel
    bg_rate_a = 800,        # Poisson background rate (cps) Acceptor channel
    )

In [ ]:
mix_sim = pbm.TimestapSimulation(S, **params)
mix_sim.summarize()

In [ ]:
rs = np.random.RandomState(1234)
mix_sim.run(rs=rs, overwrite=False, skip_existing=True)

In [ ]:
mix_sim.save_photon_hdf5()

# Burst analysis

The generated Photon-HDF5 files can be analyzed by any smFRET burst
analysis program. Here we show an example using the opensource
[FRETBursts](https://github.com/tritemio/FRETBursts/) program:

In [ ]:
import fretbursts as fb

In [ ]:
filepath = list(Path('./').glob('smFRET_*'))
filepath

In [ ]:
d = fb.loader.photon_hdf5(str(filepath[1]))

In [ ]:
d

In [ ]:
d.A_em

In [ ]:
fb.dplot(d, fb.timetrace);

In [ ]:
d.calc_bg(fun=fb.bg.exp_fit, tail_min_us='auto', F_bg=1.7)

In [ ]:
d.bg_dd, d.bg_ad

In [ ]:
d.burst_search(F=7)

In [ ]:
d.num_bursts

In [ ]:
ds = d.select_bursts(fb.select_bursts.size, th1=20)

In [ ]:
ds.num_bursts

In [ ]:
fb.dplot(d, fb.timetrace, bursts=True);

In [ ]:
fb.dplot(ds, fb.hist_fret, pdf=False)
plt.axvline(0.75);

> **NOTE:** Unless you simulated a diffusion of 30s or more the previous histogram will be very poor.

In [ ]:
fb.bext.burst_data(ds)